In [1]:
import numpy as np 
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from datetime import date

from sklearn.ensemble import RandomForestClassifier
from sklearn.externals import joblib
from sklearn.preprocessing import LabelEncoder

import pickle
import random
import time

C:\Users\011553\AppData\Local\Continuum\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [16]:
df = pd.read_csv('D:\\庫存健診開發\\data\\Training\\Raw\\Whole_new_large.csv', converters={'ts': str, 'StockNo': str, 'StockName': str})
df['ts'] = pd.to_datetime(df['ts'])
df

,ts,StockNo,StockName,open,high,low,close,return,volatility,vol,...,investment_pvt,dealer_pvt,pvt,RSI_15,SOk5,SOd3,cluster,foreign_ratio,investment_ratio,corporation_ratio
0,2007-08-14,1452,宏益,11.60,12.00,11.55,11.65,0.43,3.88,1623.0,...,0.000000,1.244881,2512.066540,0.610942,0.210413,0.457851,0,1.35,0.0,1.36
1,2007-08-15,1452,宏益,11.40,12.00,11.00,11.00,-5.58,8.58,1218.0,...,0.000000,0.971385,1995.293671,0.502573,0.413793,0.435822,0,1.35,0.0,1.36
2,2007-08-16,1452,宏益,11.00,11.00,10.25,10.25,-6.82,6.82,1757.0,...,0.000000,0.549287,1858.482748,0.393732,0.067173,0.251498,0,1.35,0.0,1.36
3,2007-08-17,1452,宏益,10.45,10.45,9.54,9.71,-5.27,8.88,1358.0,...,0.000000,0.683526,648.063462,0.318990,0.096728,0.174113,0,1.35,0.0,1.36
4,2007-08-20,1452,宏益,10.30,10.35,10.00,10.35,6.59,3.60,833.0,...,0.000000,0.179985,350.759484,0.318990,0.267024,0.220568,0,1.37,0.0,1.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1377219,2019-09-16,9958,世紀鋼,78.00,80.60,77.10,77.50,0.91,4.56,7830.0,...,31.689587,53.577270,633.328333,0.779757,0.758739,0.755888,4,2.47,1.3,5.39
1377220,2019-09-17,9958,世紀鋼,78.90,78.90,77.10,77.30,-0.26,2.32,3050.0,...,31.689587,59.420256,576.268054,0.779757,0.564237,0.660063,4,2.22,1.3,4.97
1377221,2019-09-18,9958,世紀鋼,77.90,79.30,77.50,77.70,0.91,2.34,5940.0,...,27.040960,60.731108,633.876510,0.791769,0.582582,0.621322,4,2.06,1.3,4.88
1377222,2019-09-19,9958,世紀鋼,78.40,78.40,76.20,76.30,-1.80,2.83,4011.0,...,27.144803,63.106589,562.169483,0.658399,0.173262,0.397292,4,2.08,1.3,4.82


In [6]:
27157/2601

10.440984236831987

In [17]:
df[['ts', 'StockNo', 'Y', 'VWAP', 'VWAP_return','vol', 'total']].iloc[:50]

,ts,StockNo,Y,VWAP,VWAP_return,vol,total
0,2007-08-14,1452,0.0,11.710413,-3.61,1623.0,19006.0
1,2007-08-15,1452,0.0,11.620690,-0.77,1218.0,14154.0
2,2007-08-16,1452,0.0,10.394422,-10.55,1757.0,18263.0
3,2007-08-17,1452,1.0,9.816642,-5.56,1358.0,13331.0
4,2007-08-20,1452,1.0,10.196879,3.87,833.0,8494.0
5,2007-08-21,1452,0.0,10.293706,0.95,715.0,7360.0
6,2007-08-22,1452,1.0,10.153094,-1.37,307.0,3117.0
7,2007-08-23,1452,0.0,10.573832,4.14,535.0,5657.0
8,2007-08-24,1452,1.0,10.548831,-0.24,727.0,7669.0
9,2007-08-27,1452,1.0,10.580442,0.30,317.0,3354.0


In [285]:
def label(row):

    if row['VWAP_after'] - row['VWAP_day5'] > 0:
        return 1
    elif row['VWAP_after'] - row['VWAP_day5'] <= 0:
        return 0
    
df_list = [group[1] for group in df.groupby(df['StockNo'])]
output_list = []

for i, d in enumerate(tqdm(df_list)):
    d['VWAP_after'] = d['VWAP_day5'].shift(-5)
    d['Y_weekly'] = d.apply(label, axis=1)
    output_list.append(d)
    
df = pd.concat(output_list, axis=0)
df

,ts,StockNo,StockName,open,high,low,close,return,volatility,vol,...,dealer_pvt,pvt,RSI_15,SOk5_1,SOd3,SOk15_5,SOd5,cluster,VWAP_after,Y_weekly
0,2007-08-14,2457,飛宏,36.40,36.75,34.75,34.75,-4.01,5.52,9280.0,...,423.092820,5081.873947,0.426625,0.119751,0.233466,0.222587,0.260711,0,30.142231,0.0
1,2007-08-15,2457,飛宏,33.55,34.00,32.35,32.35,-6.91,4.75,9388.0,...,496.137932,4129.096402,0.335596,0.064701,0.149083,0.288347,0.269923,0,29.653033,0.0
2,2007-08-16,2457,飛宏,31.00,31.50,30.10,30.10,-6.96,4.33,10025.0,...,500.679620,3518.651439,0.273150,0.033439,0.091261,0.244400,0.261415,0,29.895790,0.0
3,2007-08-17,2457,飛宏,29.00,30.10,28.00,28.00,-6.98,6.98,15935.0,...,529.910289,2432.983888,0.227914,0.037659,0.064460,0.215992,0.246274,0,30.972931,0.0
4,2007-08-20,2457,飛宏,29.80,29.95,28.80,29.95,6.96,4.11,10339.0,...,541.401845,1831.764140,0.227914,0.181659,0.123059,0.179349,0.223966,0,31.408509,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281155,2019-09-09,3042,晶技,40.30,40.95,40.10,40.50,1.25,2.13,2455.0,...,57.383813,387.643826,0.659964,0.211161,0.307350,0.706126,0.703066,2,NaN,NaN
281156,2019-09-10,3042,晶技,40.70,40.90,40.10,40.55,0.12,1.98,1934.0,...,57.420638,393.944625,0.659964,0.295270,0.301310,0.627819,0.677984,2,NaN,NaN
281157,2019-09-11,3042,晶技,40.85,40.90,40.20,40.20,-0.86,1.73,1284.0,...,56.914445,389.359238,0.659964,0.285293,0.293301,0.606306,0.654091,2,NaN,NaN
281158,2019-09-12,3042,晶技,40.40,40.45,40.20,40.35,0.37,0.62,1162.0,...,27.033052,688.788596,0.659964,0.256741,0.275021,0.567215,0.625133,2,NaN,NaN


In [18]:
def TrainTestSplit(data, TrainDate, valDate):

    train_df = data[data['ts'].dt.date < TrainDate][data['pvt'].notnull()]
    val_df = data[data['ts'].dt.date < valDate][data['ts'].dt.date >= TrainDate][data['pvt'].notnull()]
    test_df = data[data['ts'].dt.date >= valDate][data['pvt'].notnull()]
    
    train_df, val_df, test_df = train_df.fillna(0), val_df.fillna(0), test_df.fillna(0)
    return train_df, val_df, test_df

In [19]:
len(df[df.Y == 1])/len(df)

0.4933438569179741

In [20]:
# le = LabelEncoder()
# df['industry'] = le.fit_transform(df['industry'])
train_df, val_df, test_df = TrainTestSplit(df, date(2016,1,1), date(2017,9,1))

C:\Users\011553\AppData\Local\Continuum\anaconda3\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\011553\AppData\Local\Continuum\anaconda3\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.
C:\Users\011553\AppData\Local\Continuum\anaconda3\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


In [22]:
feature_list = ['total', 'capital', 
       'close_return', 'VWAP_return', 'sd_15', 'VWAP_return_ratio_day5',
       'return_ratio_day5', 'vol_ratio_day5', 'VWAP_ratio_day5',
       'VWAP_return_ratio_day10', 'return_ratio_day10', 'vol_ratio_day10',
       'VWAP_ratio_day10','VWAP_return_ratio_day15',
       'return_ratio_day15', 'vol_ratio_day15', 'VWAP_ratio_day15',
       'VWAP_return_ratio_day20', 'return_ratio_day20', 'vol_ratio_day20',
       'VWAP_ratio_day20', 'VWAP_return_momentum_day1',
       'return_momentum_day1', 'vol_momentum_day1', 'VWAP_momentum_day1',
       'VWAP_return_momentum_day2', 'return_momentum_day2',
       'vol_momentum_day2', 'VWAP_momentum_day2', 'VWAP_return_momentum_day3',
       'return_momentum_day3', 'vol_momentum_day3', 'VWAP_momentum_day3',
       'VWAP_return_momentum_day4', 'return_momentum_day4',
       'vol_momentum_day4', 'VWAP_momentum_day4', 'VWAP_return_momentum_day5',
       'return_momentum_day5', 'vol_momentum_day5', 'VWAP_momentum_day5',
       'VWAP_return_momentum_day10', 'return_momentum_day10',
       'vol_momentum_day10', 'VWAP_momentum_day10',
       'VWAP_return_momentum_day15', 'return_momentum_day15',
       'vol_momentum_day15', 'VWAP_momentum_day15',
       'VWAP_return_momentum_day20', 'return_momentum_day20',
       'vol_momentum_day20', 'VWAP_momentum_day20', 'vol_VWAP_corr',
       'vol_volatile_corr', 'vol_skew', 'vol_kurtosis', 'VWAP_skew', 'VWAP_kurtosis',
        'pvt', 'ATR', 'RSI_15', 'SOk5', 'SOd3', 
       'index_open', 'index_high', 'index_low', 'index_close', 'index_vol',
       'index_return', 'industry_return',  'index_VWAP_slope', 'industry_VWAP_slope', 'index_close_slope',
       'industry_close_slope', 'index_close_ratio_day5',
       'index_vol_ratio_day5', 'index_return_ratio_day5',
       'industry_close_ratio_day5', 'industry_vol_ratio_day5',
       'industry_return_ratio_day5', 'index_close_slope_ratio_day5',
       'index_VWAP_slope_ratio_day5', 'industry_close_slope_ratio_day5',
       'industry_VWAP_slope_ratio_day5', 'index_close_ratio_day10', 'index_vol_ratio_day10',
       'index_return_ratio_day10', 'industry_close_ratio_day10', 'industry_vol_ratio_day10', 'industry_return_ratio_day10',
       'index_close_slope_ratio_day10', 'index_VWAP_slope_ratio_day10',
       'industry_close_slope_ratio_day10', 'industry_VWAP_slope_ratio_day10',
      'index_close_ratio_day15', 'index_vol_ratio_day15',
       'index_return_ratio_day15', 'industry_close_ratio_day15',
       'industry_vol_ratio_day15', 'industry_return_ratio_day15',
       'index_close_slope_ratio_day15', 'index_VWAP_slope_ratio_day15',
       'industry_close_slope_ratio_day15', 'industry_VWAP_slope_ratio_day15',
      'index_close_ratio_day20', 'index_vol_ratio_day20',
       'index_return_ratio_day20', 'industry_close_ratio_day20',
       'industry_vol_ratio_day20', 'industry_return_ratio_day20',
       'index_close_slope_ratio_day20', 'index_VWAP_slope_ratio_day20',
       'industry_close_slope_ratio_day20', 'industry_VWAP_slope_ratio_day20',
      'index_close_momentum_day1', 'index_vol_momentum_day1', 'index_return_momentum_day1',
       'industry_close_momentum_day1', 'industry_vol_momentum_day1',
       'industry_return_momentum_day1', 'index_close_slope_momentum_day1',
       'index_VWAP_slope_momentum_day1', 'industry_close_slope_momentum_day1',
       'industry_VWAP_slope_momentum_day1',  'index_close_momentum_day2', 'index_vol_momentum_day2',
       'index_return_momentum_day2', 'industry_close_momentum_day2',
       'industry_vol_momentum_day2', 'industry_return_momentum_day2',
       'index_close_slope_momentum_day2', 'index_VWAP_slope_momentum_day2', 'industry_close_slope_momentum_day2',
       'industry_VWAP_slope_momentum_day2', 'index_close_momentum_day3', 'index_vol_momentum_day3',
       'index_return_momentum_day3', 'industry_close_momentum_day3',
       'industry_vol_momentum_day3', 'industry_return_momentum_day3',
       'index_close_slope_momentum_day3', 'index_VWAP_slope_momentum_day3',
       'industry_close_slope_momentum_day3', 'industry_VWAP_slope_momentum_day3', 
       'index_close_momentum_day4', 'index_vol_momentum_day4',
       'index_return_momentum_day4', 'industry_close_momentum_day4',
       'industry_vol_momentum_day4', 'industry_return_momentum_day4',
       'index_close_slope_momentum_day4', 'index_VWAP_slope_momentum_day4',
       'industry_close_slope_momentum_day4', 'industry_VWAP_slope_momentum_day4', 
        'index_close_momentum_day5', 'index_vol_momentum_day5',
       'index_return_momentum_day5', 'industry_close_momentum_day5',
       'industry_vol_momentum_day5', 'industry_return_momentum_day5',
       'index_close_slope_momentum_day5', 'index_VWAP_slope_momentum_day5',
       'industry_close_slope_momentum_day5', 'industry_VWAP_slope_momentum_day5', 
       'index_close_momentum_day10', 'index_vol_momentum_day10',
       'index_return_momentum_day10', 'industry_close_momentum_day10',
       'industry_vol_momentum_day10', 'industry_return_momentum_day10',
       'index_close_slope_momentum_day10', 'index_VWAP_slope_momentum_day10',
       'industry_close_slope_momentum_day10',
       'industry_VWAP_slope_momentum_day10', 'index_close_momentum_day15', 'index_vol_momentum_day15',
       'index_return_momentum_day15', 'industry_close_momentum_day15',
       'industry_vol_momentum_day15', 'industry_return_momentum_day15',
       'index_close_slope_momentum_day15', 'index_VWAP_slope_momentum_day15',
       'industry_close_slope_momentum_day15', 'industry_VWAP_slope_momentum_day15', 
       'index_close_momentum_day20', 'index_vol_momentum_day20',
       'index_return_momentum_day20', 'industry_close_momentum_day20',
       'industry_vol_momentum_day20', 'industry_return_momentum_day20',
       'index_close_slope_momentum_day20', 'index_VWAP_slope_momentum_day20',
       'industry_close_slope_momentum_day20', 'industry_VWAP_slope_momentum_day20', 'index_vol_close_corr',
       'index_vol_volatile_corr', 'industry_vol_close_corr',
       'industry_vol_volatile_corr', 'index_close_skew',
       'index_close_kurtosis', 'index_vol_skew', 'index_vol_kurtosis',
       'industry_close_skew', 'industry_close_kurtosis', 'industry_vol_skew',
       'industry_vol_kurtosis', 'index_pvt', 'industry_pvt',
        'foreign_buy_ratio', 'foreign_buy_day3_ratio', 'foreign_buy_week_ratio',
       'foreign_buy_2week_ratio', 'foreign_buy_3week_ratio',
       'foreign_buy_continuous', 'foreign_buy_3week_skew',
       'foreign_buy_3week_kurtosis', 'foreign_buy_week_skew',
       'foreign_buy_week_kurtosis', 'foreign_buy_2week_skew',
       'foreign_buy_2week_kurtosis', 'foreign_buy_3day_skew',
       'foreign_buy_3day_kurtosis', 'foreign_buy_buy_day_ratio',
       'foreign_buy_sell_day_ratio', 'investment_buy_ratio',
       'investment_buy_day3_ratio', 'investment_buy_week_ratio',
       'investment_buy_2week_ratio', 'investment_buy_3week_ratio',
       'investment_buy_continuous', 'investment_buy_3week_skew',
       'investment_buy_3week_kurtosis', 'investment_buy_week_skew',
       'investment_buy_week_kurtosis', 'investment_buy_2week_skew',
       'investment_buy_2week_kurtosis', 'investment_buy_3day_skew',
       'investment_buy_3day_kurtosis', 'investment_buy_buy_day_ratio',
       'investment_buy_sell_day_ratio', 'dealer_buy_ratio',
       'dealer_buy_day3_ratio', 'dealer_buy_week_ratio',
       'dealer_buy_2week_ratio', 'dealer_buy_3week_ratio',
       'dealer_buy_continuous', 'dealer_buy_3week_skew',
       'dealer_buy_3week_kurtosis', 'dealer_buy_week_skew',
       'dealer_buy_week_kurtosis', 'dealer_buy_2week_skew',
       'dealer_buy_2week_kurtosis', 'dealer_buy_3day_skew',
       'dealer_buy_3day_kurtosis', 'dealer_buy_buy_day_ratio',
       'dealer_buy_sell_day_ratio', 'foreign_pvt',
       'investment_pvt', 'dealer_pvt', 'foreign_ratio', 'investment_ratio', 'corporation_ratio', 'industry', 'ts', 'StockNo', 'Y'
       ]

In [10]:
feature_list = ['total', 'capital', 'index_open', 'index_high', 'index_low', 'index_close',
       'index_return', 'index_vol', 'industry_return',
       'foreign_ratio',
       'investment_ratio', 'corporation_ratio', 'close_return', 'VWAP_return',
       'index_VWAP_slope', 'VWAP_day5_return', 'VWAP_day5_return_5',
       'industry_VWAP_slope', 'index_close_slope', 'industry_close_slope',
       'index_close_ratio_day5', 'index_vol_ratio_day5',
       'index_return_ratio_day5', 'industry_close_ratio_day5',
       'industry_vol_ratio_day5', 'industry_return_ratio_day5',
       'VWAP_return_ratio_day5', 'close_return_ratio_day5', 'vol_ratio_day5',
       'VWAP_ratio_day5', 'index_close_ratio_day10', 'index_vol_ratio_day10',
       'index_return_ratio_day10', 'industry_close_ratio_day10',
       'industry_vol_ratio_day10', 'industry_return_ratio_day10',
       'VWAP_return_ratio_day10', 'close_return_ratio_day10',
       'vol_ratio_day10', 'VWAP_ratio_day10', 'index_close_ratio_day15',
       'index_vol_ratio_day15', 'index_return_ratio_day15',
       'industry_close_ratio_day15', 'industry_vol_ratio_day15',
       'industry_return_ratio_day15', 'VWAP_return_ratio_day15',
       'close_return_ratio_day15', 'vol_ratio_day15', 'VWAP_ratio_day15',
       'index_close_momentum_day1', 'index_vol_momentum_day1',
       'index_return_momentum_day1', 'industry_close_momentum_day1',
       'industry_vol_momentum_day1', 'industry_return_momentum_day1',
       'VWAP_return_momentum_day1', 'close_return_momentum_day1',
       'vol_momentum_day1', 'VWAP_momentum_day1', 'index_close_momentum_day2',
       'index_vol_momentum_day2', 'index_return_momentum_day2',
       'industry_close_momentum_day2', 'industry_vol_momentum_day2',
       'industry_return_momentum_day2', 'VWAP_return_momentum_day2',
       'close_return_momentum_day2', 'vol_momentum_day2', 'VWAP_momentum_day2',
       'index_close_momentum_day3', 'index_vol_momentum_day3',
       'index_return_momentum_day3', 'industry_close_momentum_day3',
       'industry_vol_momentum_day3', 'industry_return_momentum_day3',
       'VWAP_return_momentum_day3', 'close_return_momentum_day3',
       'vol_momentum_day3', 'VWAP_momentum_day3',
       'index_close_momentum_day4', 'index_vol_momentum_day4',
       'index_return_momentum_day4', 'industry_close_momentum_day4',
       'industry_vol_momentum_day4', 'industry_return_momentum_day4',
       'VWAP_return_momentum_day4', 'close_return_momentum_day4',
       'vol_momentum_day4', 'VWAP_momentum_day4', 'index_close_momentum_day5',
       'index_vol_momentum_day5', 'index_return_momentum_day5',
       'industry_close_momentum_day5', 'industry_vol_momentum_day5',
       'industry_return_momentum_day5', 'VWAP_return_momentum_day5',
       'close_return_momentum_day5', 'vol_momentum_day5', 'VWAP_momentum_day5',
       'index_close_momentum_day10', 'index_vol_momentum_day10',
       'index_return_momentum_day10', 'industry_close_momentum_day10',
       'industry_vol_momentum_day10', 'industry_return_momentum_day10',
       'VWAP_return_momentum_day10', 'close_return_momentum_day10',
       'vol_momentum_day10', 'VWAP_momentum_day10', 'index_close_momentum_day15',
       'index_vol_momentum_day15', 'index_return_momentum_day15',
       'industry_close_momentum_day15', 'industry_vol_momentum_day15',
       'industry_return_momentum_day15', 'VWAP_return_momentum_day15',
       'close_return_momentum_day15', 'vol_momentum_day15',
       'VWAP_momentum_day15', 'vol_VWAP_corr', 'vol_volatile_corr',
       'index_vol_close_corr', 'index_vol_volatile_corr',
       'industry_vol_close_corr', 'industry_vol_volatile_corr',
       'index_close_skew', 'index_close_kurtosis', 'index_vol_skew',
       'index_vol_kurtosis', 'industry_close_skew', 'industry_close_kurtosis', 'industry_vol_skew',
       'industry_vol_kurtosis', 'vol_skew', 'vol_kurtosis', 'VWAP_skew',
       'VWAP_kurtosis', 'VWAP_day5_skew', 'VWAP_day5_kurtosis',
       'foreign_buy_ratio', 'foreign_buy_day3_ratio', 'foreign_buy_week_ratio',
       'foreign_buy_2week_ratio', 'foreign_buy_3week_ratio',
       'foreign_buy_continuous', 'foreign_buy_buy_indicator',
       'foreign_buy_sell_indicator', 'investment_buy_ratio',
       'investment_buy_day3_ratio', 'investment_buy_week_ratio',
       'investment_buy_2week_ratio', 'investment_buy_3week_ratio',
       'investment_buy_continuous', 'investment_buy_buy_day_ratio',
       'investment_buy_sell_day_ratio', 'dealer_buy_ratio',
       'dealer_buy_day3_ratio', 'dealer_buy_week_ratio',
       'dealer_buy_2week_ratio', 'dealer_buy_3week_ratio',
       'dealer_buy_continuous', 'dealer_buy_buy_day_ratio',
       'dealer_buy_sell_day_ratio', 'index_pvt', 'industry_pvt', 'foreign_pvt',
       'investment_pvt', 'dealer_pvt', 'pvt', 'RSI_15', 'SOk5_1', 'SOd3',
       'SOk15_5', 'SOd5', 'industry', 'ts', 'StockNo']

In [25]:
feature_list = ['total', 'capital', 
       'industry', 'index_open', 'index_high', 'index_low', 'index_close',
       'index_return', 'index_vol', 'close_return',
       'VWAP_return', 'sd_4', 'index_VWAP_slope',
       'industry_VWAP_slope', 'index_close_slope', 'industry_close_slope',
       'index_close_ratio_week1', 'index_vol_ratio_week1',
       'index_return_ratio_week1', 'industry_close_ratio_week1',
       'industry_vol_ratio_week1', 'industry_return_ratio_week1',
       'index_close_slope_ratio_week1', 'index_VWAP_slope_ratio_week1',
       'industry_close_slope_ratio_week1', 'industry_VWAP_slope_ratio_week1',
       'VWAP_return_ratio_week1', 'return_ratio_week1', 'vol_ratio_week1',
       'VWAP_ratio_week1', 'index_close_ratio_week2', 'index_vol_ratio_week2',
       'index_return_ratio_week2', 'industry_close_ratio_week2',
       'industry_vol_ratio_week2', 'industry_return_ratio_week2',
       'index_close_slope_ratio_week2', 'index_VWAP_slope_ratio_week2',
       'industry_close_slope_ratio_week2', 'industry_VWAP_slope_ratio_week2',
        'VWAP_return_ratio_week2', 'return_ratio_week2', 'vol_ratio_week2',
       'VWAP_ratio_week2', 'index_close_ratio_week3', 'index_vol_ratio_week3',
       'index_return_ratio_week3', 'industry_close_ratio_week3',
       'industry_vol_ratio_week3', 'industry_return_ratio_week3',
       'index_close_slope_ratio_week3', 'index_VWAP_slope_ratio_week3',
       'industry_close_slope_ratio_week3', 'industry_VWAP_slope_ratio_week3',
       'VWAP_return_ratio_week3', 'return_ratio_week3', 'vol_ratio_week3',
       'VWAP_ratio_week3', 'index_close_ratio_week4', 'index_vol_ratio_week4',
       'index_return_ratio_week4', 'industry_close_ratio_week4',
       'industry_vol_ratio_week4', 'industry_return_ratio_week4',
       'index_close_slope_ratio_week4', 'index_VWAP_slope_ratio_week4',
       'industry_close_slope_ratio_week4', 'industry_VWAP_slope_ratio_week4',
       'VWAP_return_ratio_week4', 'return_ratio_week4', 'vol_ratio_week4',
       'VWAP_ratio_week4', 'index_close_momentum_week1',
       'index_vol_momentum_week1', 'index_return_momentum_week1',
       'industry_close_momentum_week1', 'industry_vol_momentum_week1',
       'industry_return_momentum_week1', 'index_close_slope_momentum_week1',
       'index_VWAP_slope_momentum_week1',
       'industry_close_slope_momentum_week1',
       'industry_VWAP_slope_momentum_week1', 'VWAP_return_momentum_week1',
       'return_momentum_week1', 'vol_momentum_week1', 'VWAP_momentum_week1',
       'index_close_momentum_week2', 'index_vol_momentum_week2',
       'index_return_momentum_week2', 'industry_close_momentum_week2',
       'industry_vol_momentum_week2', 'industry_return_momentum_week2',
       'index_close_slope_momentum_week2', 'index_VWAP_slope_momentum_week2',
       'industry_close_slope_momentum_week2',
       'industry_VWAP_slope_momentum_week2', 'VWAP_return_momentum_week2',
       'return_momentum_week2', 'vol_momentum_week2', 'VWAP_momentum_week2',
        'index_close_momentum_week3', 'index_vol_momentum_week3',
       'index_return_momentum_week3', 'industry_close_momentum_week3',
       'industry_vol_momentum_week3', 'industry_return_momentum_week3',
       'index_close_slope_momentum_week3', 'index_VWAP_slope_momentum_week3',
       'industry_close_slope_momentum_week3',
       'industry_VWAP_slope_momentum_week3', 'VWAP_return_momentum_week3',
       'return_momentum_week3', 'vol_momentum_week3', 'VWAP_momentum_week3',
       'index_close_momentum_week4', 'index_vol_momentum_week4',
       'index_return_momentum_week4', 'industry_close_momentum_week4',
       'industry_vol_momentum_week4', 'industry_return_momentum_week4',
       'index_close_slope_momentum_week4', 'index_VWAP_slope_momentum_week4',
       'industry_close_slope_momentum_week4',
       'industry_VWAP_slope_momentum_week4', 'VWAP_return_momentum_week4',
       'return_momentum_week4', 'vol_momentum_week4', 'VWAP_momentum_week4',
       'vol_VWAP_corr', 'vol_volatile_corr', 'index_vol_close_corr',
       'index_vol_volatile_corr', 'industry_vol_close_corr',
       'industry_vol_volatile_corr', 'index_close_skew',
       'index_close_kurtosis', 'index_vol_skew', 'index_vol_kurtosis',
       'industry_close_skew', 'industry_close_kurtosis', 'industry_vol_skew',
       'industry_vol_kurtosis', 'vol_skew', 'vol_kurtosis', 'VWAP_skew',
       'VWAP_kurtosis', 'ATR_x', 'index_pvt_week', 'industry_pvt_week', 'MACD_diff', 'RSI_15', 'SOk5', 'SOd3',
       'pvt_week', 'RSI_4_week', 'SOk4_week', 'SOd1_week', 'foreign_buy_week_ratio', 'foreign_buy_2week_ratio',
       'foreign_buy_3week_ratio', 'foreign_buy_continuous',
       'foreign_buy_buy_day_ratio', 'foreign_buy_sell_day_ratio',
       'investment_buy_week_ratio', 'investment_buy_continuous',
       'investment_buy_buy_day_ratio', 'investment_buy_sell_day_ratio',
       'foreign_ratio', 'investment_ratio', 'corporation_ratio', 'ts', 'StockNo', 'Y_weekly']

In [23]:
X_train, Y_train = train_df[feature_list], train_df['Y']
X_val, Y_val = val_df[feature_list], val_df['Y']
# X_test, Y_test = test_df[feature_list], test_df['Y']


In [24]:
X_train

,total,capital,close_return,VWAP_return,sd_15,VWAP_return_ratio_day5,return_ratio_day5,vol_ratio_day5,VWAP_ratio_day5,VWAP_return_ratio_day10,...,foreign_pvt,investment_pvt,dealer_pvt,foreign_ratio,investment_ratio,corporation_ratio,industry,ts,StockNo,Y
0,19006.0,16.0,0.43,-3.61,0.053587,-10.373563,2.216495,0.720373,0.977153,-5.494673,...,-9.419620,0.000000,1.244881,1.35,0.00,1.36,紡織纖維,2007-08-14,1452,0.0
1,14154.0,15.1,-5.58,-0.77,0.047987,1.007853,3.278496,0.655614,0.977530,-1.469466,...,-14.722766,0.000000,0.971385,1.35,0.00,1.36,紡織纖維,2007-08-15,1452,0.0
2,18263.0,14.0,-6.82,-10.55,0.051591,3.359873,2.224397,1.019733,0.902773,37.411348,...,-15.992266,0.000000,0.549287,1.35,0.00,1.36,紡織纖維,2007-08-16,1452,0.0
3,13331.0,13.3,-5.27,-5.56,0.061742,1.672684,1.422786,0.744844,0.881356,5.656155,...,-15.714338,0.000000,0.683526,1.35,0.00,1.36,紡織纖維,2007-08-17,1452,1.0
4,8494.0,14.2,6.59,3.87,0.065569,-1.164260,-3.093897,0.613492,0.948740,-8.062500,...,-15.730436,0.000000,0.179985,1.37,0.00,1.37,紡織纖維,2007-08-20,1452,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1105886,89383.0,222.3,1.77,0.94,0.050744,1.160494,2.458333,0.558399,1.015643,1.623489,...,13.268638,222.211173,122.026386,30.72,5.94,37.79,其他,2015-12-25,9938,0.0
1105887,51475.0,219.0,-1.47,-0.99,0.043792,-1.972112,-3.049793,0.361583,1.000732,-0.920930,...,16.627583,218.449557,122.652980,30.70,5.94,37.71,其他,2015-12-28,9938,0.0
1105888,36644.0,215.2,-1.77,-1.02,0.037143,-3.128834,-8.349057,0.284839,0.987552,-1.014925,...,10.132339,219.506494,123.459515,30.71,5.91,37.69,其他,2015-12-29,9938,0.0
1105889,88897.0,216.9,0.83,-0.10,0.036931,0.227273,-4.560440,1.226852,0.990990,-0.154083,...,10.388587,219.172969,123.530345,30.65,5.96,37.66,其他,2015-12-30,9938,1.0


In [25]:
X_train.to_csv('D:\\OneDrive - SinoPac\\training\\train_daily_without_MACD.csv', index=False)
X_val.to_csv('D:\\OneDrive - SinoPac\\training\\val_daily_without_MACD.csv', index=False)
# X_test.to_csv('D:\\OneDrive - SinoPac\\training\\test_rolling.csv', index=False)

In [242]:
X_train.columns[:50]

Index(['total', 'capital', 'VWAP_return', 'industry', 'index_open',
       'index_high', 'index_low', 'index_close', 'index_return', 'index_vol',
       'VWAP_return', 'sd_4', 'index_VWAP_slope', 'industry_VWAP_slope',
       'index_close_slope', 'industry_close_slope', 'ATR_x', 'index_pvt_week',
       'industry_pvt_week', 'MACD_diff', 'RSI_15', 'SOk5', 'SOd3', 'pvt_week',
       'RSI_4_week', 'SOk4_week', 'SOd1_week', 'foreign_buy_week_ratio',
       'foreign_buy_2week_ratio', 'foreign_buy_3week_ratio',
       'foreign_buy_continuous', 'foreign_buy_buy_day_ratio',
       'foreign_buy_sell_day_ratio', 'investment_buy_week_ratio',
       'investment_buy_continuous', 'investment_buy_buy_day_ratio',
       'investment_buy_sell_day_ratio', 'foreign_ratio', 'investment_ratio',
       'corporation_ratio'],
      dtype='object')

In [48]:
RF2 = RandomForestClassifier(n_estimators=100, 
                            max_depth=20,
                            max_features=0.9,
                            min_samples_leaf=0.1, 
                            min_samples_split=0.1,
                            verbose=1, 
                            n_jobs=-1,
                            oob_score=True)

In [49]:
RF2.fit(X_train, Y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  4.8min finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=20, max_features=0.9, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=0.1, min_samples_split=0.1,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=True, random_state=None, verbose=1,
                       warm_start=False)

In [50]:
RF2.oob_score_

0.6875382977676595

In [51]:
prediction_proba = RF2.predict_proba(X_train)
prediction = RF2.predict(X_train)

[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.2s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.9s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.2s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.9s finished


In [32]:
def predict(prediction, threshold):
    predict = []
    for item in prediction:
        if item[1] > threshold:
            predict.append(1)
        else:
            predict.append(0)
            
    return predict

In [33]:
new_prediction = predict(prediction_proba.tolist(), threshold = 0.7)
new_prediction.count(1)/len(new_prediction)

0.20969022414204508

In [52]:
from sklearn.metrics import classification_report
target_names = ['down', 'up']
print(classification_report(Y_train, prediction, target_names=target_names))

              precision    recall  f1-score   support

        down       0.69      0.70      0.69    448386
          up       0.69      0.68      0.68    445922

    accuracy                           0.69    894308
   macro avg       0.69      0.69      0.69    894308
weighted avg       0.69      0.69      0.69    894308



In [35]:
from sklearn.metrics import classification_report
target_names = ['down', 'up']
print(classification_report(Y_train, new_prediction, target_names=target_names))

              precision    recall  f1-score   support

        down       0.58      0.92      0.71    450307
          up       0.80      0.34      0.47    446535

    accuracy                           0.63    896842
   macro avg       0.69      0.63      0.59    896842
weighted avg       0.69      0.63      0.59    896842



In [208]:
X_train

,total,capital,industry,index_open,index_high,index_low,index_close,index_return,index_vol,close_return,...,foreign_buy_continuous,foreign_buy_buy_day_ratio,foreign_buy_sell_day_ratio,investment_buy_week_ratio,investment_buy_continuous,investment_buy_buy_day_ratio,investment_buy_sell_day_ratio,foreign_ratio,investment_ratio,corporation_ratio
1972,5419980.0,1169.5,5,9030.82,9030.93,7987.61,8090.29,-9.42,27495607,-8.75,...,-7,0.200000,0.800000,0.100781,1,0.800000,0.200000,44.93,2.27,47.40
1973,5967624.0,1293.0,5,8496.97,8741.64,8397.26,8690.09,7.41,22396974,10.56,...,3,0.333333,0.666667,0.032144,2,0.733333,0.266667,44.80,2.42,47.48
1974,4664043.0,1377.4,5,8849.96,8982.16,8514.47,8982.16,3.36,21385988,6.53,...,3,0.466667,0.533333,-0.016999,1,0.533333,0.466667,45.54,2.36,48.24
1975,3298731.0,1349.8,5,9018.89,9078.47,8841.64,9018.08,0.40,21444798,-2.00,...,-1,0.666667,0.333333,0.044350,2,0.466667,0.533333,45.82,2.47,48.58
1976,3196760.0,1374.2,5,8868.00,9084.97,8845.90,9031.63,0.15,20331222,1.81,...,2,0.600000,0.400000,-0.061375,1,0.400000,0.600000,46.00,2.32,48.66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286154,1408316.0,619.8,0,8339.83,8497.19,8239.00,8398.60,0.00,23294485,1.14,...,-1,0.400000,0.600000,-0.000220,-1,0.466667,0.133333,15.73,0.28,16.21
286155,1105678.0,605.9,0,8432.47,8535.36,8108.68,8115.89,-3.37,22101134,-2.25,...,1,0.466667,0.533333,0.000000,0,0.400000,0.133333,15.86,0.28,16.27
286156,701771.0,605.2,0,8058.67,8319.67,7980.59,8257.32,1.74,21747693,-0.11,...,-1,0.533333,0.466667,0.023691,3,0.400000,0.200000,15.82,0.31,16.23
286157,493160.0,611.5,0,8228.05,8393.41,8191.53,8363.28,1.28,20148728,1.04,...,-1,0.733333,0.266667,0.003021,0,0.266667,0.066667,15.98,0.31,16.38


In [21]:
test_df['Y_1_score'] = [item[1] for item in prediction_proba.tolist()]

In [40]:
d = test_df[test_df['corporation_ratio'] < 22]

In [41]:
len(d[(d['Y_1_score'] > 0.5) & (d['Y_weekly'] == 1)])/len(d[(d['Y_1_score'] > 0.5)])

0.6898810908779192

In [138]:
RF2.feature_importances_

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       7.91604497e-04, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       9.22952022e-01, 5.83494888e-02, 0.00000000e+00, 0.00000000e+00,
       8.70438780e-03, 9.20249707e-03, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00])

In [150]:
df[df.ts.dt.weekday != 4]['ts'].unique()

array(['2007-09-29T00:00:00.000000000', '2008-04-03T00:00:00.000000000',
       '2008-10-09T00:00:00.000000000', '2008-12-31T00:00:00.000000000',
       '2009-01-10T00:00:00.000000000', '2009-01-17T00:00:00.000000000',
       '2009-01-21T00:00:00.000000000', '2009-04-30T00:00:00.000000000',
       '2009-05-27T00:00:00.000000000', '2009-06-06T00:00:00.000000000',
       '2009-08-06T00:00:00.000000000', '2009-12-31T00:00:00.000000000',
       '2010-02-06T00:00:00.000000000', '2010-02-10T00:00:00.000000000',
       '2012-01-18T00:00:00.000000000', '2012-02-04T00:00:00.000000000',
       '2012-03-03T00:00:00.000000000', '2012-12-22T00:00:00.000000000',
       '2013-02-06T00:00:00.000000000', '2013-02-23T00:00:00.000000000',
       '2013-04-03T00:00:00.000000000', '2013-09-14T00:00:00.000000000',
       '2013-09-18T00:00:00.000000000', '2014-01-27T00:00:00.000000000',
       '2014-02-27T00:00:00.000000000', '2014-04-03T00:00:00.000000000',
       '2014-10-09T00:00:00.000000000', '2014-12-27

In [148]:
df.ts

0        2007-08-17
1        2007-08-24
2        2007-08-31
3        2007-09-07
4        2007-09-14
            ...    
286550   2019-08-16
286551   2019-08-23
286552   2019-08-30
286553   2019-09-06
286554   2019-09-12
Name: ts, Length: 286555, dtype: datetime64[ns]